In [ ]:
!pip install contractions spacy tqdm deep-translator

In [5]:
import json
import os
import re
import string

import contractions
import nltk
import pandas as pd
import spacy
import text2emotion as te

from deep_translator import GoogleTranslator as Translator
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mother/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mother/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mother/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
pd.set_option('display.max_columns', None)

# Sentiment Analysis

In [10]:
# Read from files
json_dir = './data-cleaned/' # later change this to data-cleaned, because there is the actual data (not sampled)

df_dict = {}
for root, dirs, files in os.walk(json_dir):
    for file_name in files:
        if file_name.endswith('.json'):
            file_path = os.path.join(root, file_name)
            name = os.path.splitext(file_name)[0]
            df_dict[name] = pd.read_json(path_or_buf=file_path, lines=True)

In [11]:
#df_dict

In [12]:
additional_stopwords = [
    "https",
    "de",
    "la",
    "‚Äô",
    "amp",
    "...",
    "``",
    "''",
    "'s",
    "der",
    "Doctor",
    "en",
    "que",
    "Worlds",
    "Marburg",
    "health",
    "Health",
    "world",
    "Worlds",
    "die",
    "und",
    "doctorwho-worldsapart.com/r/4608/",
]

In [13]:
# make this iterable for all dfs, else skipppp
all_words = [word for tokens in df['tokens'] for word in tokens]
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in all_words if word.lower() not in stop_words]
tokens_without_punctuation = [token for token in filtered_words if token not in string.punctuation]
filtered_words = [word for word in tokens_without_punctuation if word.lower() not in additional_stopwords]
filtered_freq_dist = FreqDist(filtered_words)

NameError: name 'df' is not defined

In [ ]:
# Blow out some freqDist Charts other than WordClouds

# Classify emotions

In [14]:
def analyse_emotion(text):
    return te.get_emotion(text)

# df["analyseEmotion"] = df["translatedContent"].apply(analyse_emotion)

In [15]:
for frame in df_dict:
    print(f"Currently working on: {frame}")
    df_dict[frame].fillna(value='', inplace=True)
    
    # Convert column to string type
    df_dict[frame]["analyseEmotion"] = df_dict[frame]["cleanedContent"].astype(str)

    # Classify the dataframe
    df_dict[frame]["analyseEmotion"] = df_dict[frame]["cleanedContent"].apply(
        analyse_emotion
    )
    df_dict[frame]["location"] = df_dict[frame].user.apply(lambda x: x["location"])

    # Save to files # insert a condition to check, if file exists
    df_dict[frame].to_json(
        f"./data-emotions/processed_{frame}.json", orient="records", lines=True
    )

Currently working on: processed_tw_hshtag_covid19
Currently working on: processed_tw_hshtag_monkeypox


In [16]:
frame

'processed_tw_hshtag_monkeypox'

In [ ]:
df_dict[frame]["location"]

In [ ]:
df_dict[frame]["location"].value_counts()